In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "flight_aware_copa_2.dump"

In [16]:
#lecture des jsons
for chunk in pd.read_json(file, chunksize=1000000, lines=True):
    first_line = pd.concat([first_line, chunk])

In [22]:
#visulalisation des données brutes
#print(first_line.dtypes)
#print(first_line.shape)
first_line[first_line.type == "arrival"].head(100).T #retourne les 100 premières lignes+transposée pour apercu

(268178, 32)


In [28]:
# Extraction des Data flightplan + nettoyage
flightplan = first_line[first_line.type == "flightplan"] #selection de tous les flightplans
flightplan = flightplan[flightplan.facility_name != "FlightAware Estimated"] #supression des flightplans estimés
flightplan = flightplan.drop_duplicates(['id'], keep='last') #prendre uniquement le dernier plan de vol envoyé

In [32]:
# Enlever les vols annulés
cancellation = first_line[first_line.type == "cancellation"]
cancellation_id = cancellation["id"].tolist()
#print(cancellation_id)
flightplan = flightplan[~flightplan.id.isin(cancellation_id)] #suppression des vols annulés
#print(flightplan.shape)

In [33]:
# Sélection des paramètres pour les flightplans + nettoyage
list_donnees_flightplan = ['id','edt','eta','orig','dest','waypoints'] #donnees_list contient les noms des colonnes à conserver
flightplan = flightplan.loc[:,list_donnees_flightplan] #on ne garde que les attributs qui nous interessent
flightplan = flightplan[flightplan.dest.notnull()]
flightplan = flightplan[flightplan.orig.notnull()]
flightplan = flightplan[flightplan.waypoints.notnull()]

print(flightplan.shape)
flightplan[flightplan.id == flightplan.id].head(100).T #apercu

(391, 6)


,1407,2013,6694,6967,7083,7247,7346,9540,10033,10301,...,47408,47409,47517,49251,50910,51056,51835,52526,52898,52901
id,CMP248-1487312750-airline-0016,CMP277-1487312750-airline-0199,CMP807-1487398200-schedule-0000,CMP495-1487391780-schedule-0000,CMP209-1487312750-airline-0140,CMP459-1487499343-0-1-32,CMP276-1487392920-schedule-0000,CMP465-1487504445-3-0-94,CMP362-1487312750-airline-0270,CMP174-1487398800-schedule-0000,...,CMP305-1487399176-airline-0050,CMP135-1487399175-airline-0246,CMP227-1487399176-airline-0260,CMP143-1487399175-airline-0052,CMP450-1487399176-airline-0110,CMP168-1487399175-airline-0147,CMP498-1487399176-airline-0266,CMP162-1487556945-0-0-46,CMP111-1487399175-airline-0150,CMP138-1487399175-airline-0204
edt,1.48756e+09,1.48756e+09,1.48757e+09,1.48756e+09,1.48756e+09,1.48759e+09,1.48757e+09,1.48759e+09,1.48757e+09,1.48757e+09,...,1.48762e+09,1.48763e+09,1.48763e+09,1.48763e+09,1.48764e+09,1.48764e+09,1.48763e+09,1.48765e+09,1.48763e+09,1.48763e+09
eta,1.48758e+09,1.48758e+09,1.48759e+09,1.48759e+09,1.48759e+09,1.48759e+09,1.48759e+09,1.48759e+09,1.48759e+09,1.4876e+09,...,1.48764e+09,1.48764e+09,1.48764e+09,1.48764e+09,1.48765e+09,1.48764e+09,1.48765e+09,1.48765e+09,1.48765e+09,1.48765e+09
orig,MPTO,MPTO,KJFK,SCEL,KSFO,MROC,SCEL,MROC,KLAX,SCEL,...,KIAD,MMMX,KMIA,TJSJ,MPTO,MNMG,MPTO,MPTO,MPTO,MPTO
dest,SCEL,SCEL,MPTO,MPTO,MPTO,MPTO,MPTO,MPTO,MPTO,MPTO,...,MPTO,MPTO,MPTO,MPTO,TJSJ,MROC,KMSY,MROC,SCEL,MMMX
waypoints,"[{'lat': 9.07, 'lon': -79.38}, {'lat': 8.97, '...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 8.97, '...","[{'lat': 40.64, 'lon': -73.78}, {'lat': 40.59,...","[{'lat': -33.38, 'lon': -70.78}, {'lat': -33.2...","[{'lat': 37.62, 'lon': -122.38}, {'lat': 37.61...","[{'lat': 10.0, 'lon': -84.22}, {'lat': 9.98, '...","[{'lat': -33.38, 'lon': -70.78}, {'lat': -33.2...","[{'lat': 10.0, 'lon': -84.22}, {'lat': 9.98, '...","[{'lat': 33.94, 'lon': -118.41}, {'lat': 33.95...","[{'lat': -33.38, 'lon': -70.78}, {'lat': -33.2...",...,"[{'lat': 38.95, 'lon': -77.46}, {'lat': 38.88,...","[{'lat': 19.43, 'lon': -99.07}, {'lat': 19.39,...","[{'lat': 25.8, 'lon': -80.29}, {'lat': 25.77, ...","[{'lat': 18.44, 'lon': -66.0}, {'lat': 18.45, ...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 10.24, ...","[{'lat': 12.15, 'lon': -86.17}, {'lat': 12.14,...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 10.6, '...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 9.09, '...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 8.97, '...","[{'lat': 9.07, 'lon': -79.38}, {'lat': 9.11, '..."


In [34]:
# Extraction des Data position + nettoyage
position = first_line[first_line.type == "position"] #selection de toutes les positions
#print(position.groupby('updateType').size()) #etude des differents types de "updateType"
position = position[position.updateType != "P"] #supression des positions estimés

In [36]:
# Enlever vols en cours
arrival = first_line[first_line.type == "arrival"]
arrival_id = arrival["id"].tolist()
position = position[position.id.isin(arrival_id)] #enlever les vols en cours

In [37]:
# Sélection des paramètres pour les positions + nettoyage
list_donnees_position=['id','clock','lat','lon','alt'] #donnees_list contient les noms des colonnes à conserver
position = position.loc[:,list_donnees_position] 
position = position[position.lat.notnull()]
position = position[position.lon.notnull()]
position = position[position.alt >= 10000] # prendre les points qu'en route (alt>100 FL)

print(position.shape)
position[position.id == position.id].head(100).T #retourne les 100 premières lignes+transposée pour apercu

(85175, 5)


,10,12,13,14,15,16,17,18,20,21,...,212,215,216,217,218,220,221,222,223,225
id,CMP872-1487392500-schedule-0000,CMP277-1487312750-airline-0199,CMP284-1487391960-schedule-0000,CMP209-1487312750-airline-0140,CMP303-1487312750-airline-0145,CMP277-1487312750-airline-0199,CMP216-1487312750-airline-0128,CMP204-1487394900-schedule-0000,CMP820-1487392500-schedule-0000,CMP276-1487392920-schedule-0000,...,CMP495-1487391780-schedule-0000,CMP872-1487392500-schedule-0000,CMP807-1487398200-schedule-0000,CMP284-1487391960-schedule-0000,CMP209-1487312750-airline-0140,CMP759-1487312751-airline-0220,CMP303-1487312750-airline-0145,CMP204-1487394900-schedule-0000,CMP216-1487312750-airline-0128,CMP277-1487312750-airline-0199
clock,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,...,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09,1.48758e+09
lat,-9.16269,-28.6843,-17.4975,19.6122,31.8173,-28.7387,-15.8451,-8.1373,-10.0248,-13.3103,...,-10.2346,-8.84436,28.4827,-17.0956,19.327,-18.35,31.4892,-7.84126,-16.1221,-29.1114
lon,-63.602,-72.6751,-65.3224,-98.8813,-114.548,-72.635,-52.6228,-63.77,-64.1799,-77.6873,...,-78.5326,-63.9244,-78.404,-65.5767,-98.5577,-49.6254,-114.109,-64.1192,-52.2757,-72.3574
alt,34000,35000,36000,35000,35000,35000,37000,38000,36000,38000,...,36000,34000,34000,36000,35000,39000,35000,38000,37000,35000


In [38]:
#sauvegarde des données nettoyées
flightplan.to_pickle("flightplan.pkl")
position.to_pickle("position.pkl")

In [ ]:
#pour récupérer les dataframes nettoyés
#flightplan = pd.read_pickle("flightplan.pkl")
#position = pd.read_pickle("position.pkl")